# Data Science Example

In [1]:
# dependencies
import psycopg2
import pandas as pd
import xlrd
import urllib

In [2]:
try:
    conn = psycopg2.connect("dbname='gpadmin' user='gpadmin' host='gpdb' password='pivotal'")
    cur = conn.cursor()
    conn.autocommit = True
except:
    print "I am unable to connect to the database"

In [ ]:
# download xlsx file
dls = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
urllib.urlretrieve(dls, "OnlineRetail.xlsx")

In [65]:
# create table shell in gpdb
query = """
    DROP TABLE IF EXISTS public.online_retail_data
"""
cur.execute(query)

ddl = """
    CREATE TABLE public.online_retail_data (
        invoiceno text
       ,stockcode text
       ,description text
       ,quantity float
       ,invoicedate timestamp
       ,unitprice float
       ,customerid text
       ,country text
    ) DISTRIBUTED BY (invoiceno);
"""
cur.execute(ddl)

In [66]:
# check if string contains all acsii characters
def is_ascii(s):
    if isinstance(s, float) or isinstance(s, int):
        return True
    else:
        return all(ord(c) < 128 for c in s)

In [67]:
def xlsx_timestamp_to_gpdb(dt):
    d = xlrd.xldate_as_tuple(dt,0)
    return "{}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}".format(d[0],d[1],d[2],d[3],d[4],d[5])

In [68]:
# function to insert records into existing table
def create_row(types, record):
    rowValues = '';
        
    for i in range(0,len(types)):
        
        # handle missing and non-ascii strings -> create some nulls to handle in example 
        #(could also change table type to handle non ascii)
        if record[i] == '' or is_ascii(record[i]) == False:
            rowValues = rowValues + ", null"
        else:
            # add single quotes around text fields
            if types[i] in ['text','varchar']:
                rowValues = rowValues + ",$r${}$r$".format(str(record[i]))
            elif types[i] == 'timestamp':
                rowValues = rowValues + ",'{}'".format(xlsx_timestamp_to_gpdb(record[i])) 
            else:
                rowValues = rowValues + ",{}".format(record[i])
            
    return ",({})".format(rowValues[1:])

In [ ]:
# load xlsx file into gpdb
def load_xlsx_file(xlsxFilename):
    
    # open workbook
    workbook = xlrd.open_workbook(xlsxFilename)
    sheet = workbook.sheet_by_index(0)
    
    prep_data = ''
    
    # iterate through rows in sheet
    for r in range(1,sheet.nrows):
        
        rec = sheet.row_values(r)
        
        colTypes = ["text","text","text","float","timestamp","float","text","text"]
        
        prep_data = prep_data + create_row(colTypes,rec)
        
        # batch load N records at a time
        if (r % 1000) == 0:  
            query = """
                INSERT INTO public.online_retail_data VALUES {}
            """.format(prep_data[1:])
            cur.execute(query)
            prep_data = ''
    
    # load any remaining record
    if prep_data != '':
        query = """
              INSERT INTO public.online_retail_data VALUES {}
        """.format(prep_data[1:])
        cur.execute(query)       

    # clean up
    workbook.release_resources()
    del workbook
    
load_xlsx_file("OnlineRetail.xlsx")

In [ ]:
# parse cancellation field out of invoiceno
query = """
    DROP TABLE IF EXISTS public.online_retail_data_2;
    CREATE TABLE public.online_retail_data_2 AS
    SELECT CASE WHEN substring(invoiceno,1,1) = 'C' THEN 1 ELSE 0 END AS cancellation
          ,CASE WHEN substring(invoiceno,1,1) = 'C' THEN substring(invoiceno,2,6)
                ELSE substring(invoiceno,1,6)
           END AS invoiceno
          ,stockcode 
          ,description 
          ,quantity 
          ,invoicedate
          ,unitprice 
          ,customerid 
          ,country 
    FROM public.online_retail_data
    DISTRIBUTED BY (invoiceno);
    DROP TABLE public.online_retail_data;
    ALTER TABLE public.online_retail_data_2 RENAME TO online_retail_data;
"""

cur.execute(query)

In [ ]:
query = """
    SELECT cancellation
          ,invoiceno
          ,stockcode 
          ,description 
          ,quantity 
          ,invoicedate
          ,unitprice 
          ,customerid 
          ,country 
    FROM public.online_retail_data
    LIMIT 10
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

In [13]:
query = """
    SELECT count(*)
    FROM public.online_retail_data
"""
cur.execute(query)
pd.DataFrame(cur.fetchall())

,0
0,541000
